In [1]:
!pip install "ray[tune]" torch torchvision pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 37.6 MB/s eta 0:00:00


In [2]:
import math

import torch
import pytorch_lightning as pl
from filelock import FileLock
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import transforms
import os

In [7]:
class LightningMNISTClassifier(pl.LightningModule):

    def __init__(self, config, data_dir=None):
        super(LightningMNISTClassifier, self).__init__()

        self.data_dir = data_dir or os.getcwd()

        self.layer_1_size = config["layer_1_size"]
        self.layer_2_size = config["layer_2_size"]
        self.lr = config["lr"]
        self.batch_size = config["batch_size"]

        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, self.layer_1_size)
        self.layer_2 = torch.nn.Linear(self.layer_1_size, self.layer_2_size)
        self.layer_3 = torch.nn.Linear(self.layer_2_size, 10)

    def forward(self, x):
        batch_size, channels, width, height = x.size()
        x = x.view(batch_size, -1)

        x = self.layer_1(x)
        x = torch.relu(x)

        x = self.layer_2(x)
        x = torch.relu(x)

        x = self.layer_3(x)
        x = torch.log_softmax(x, dim=1)

        return x

    def cross_entropy_loss(self, logits, labels):
        return F.nll_loss(logits, labels)

    def accuracy(self, logits, labels):
        _, predicted = torch.max(logits.data, 1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / len(labels)
        return torch.tensor(accuracy)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        accuracy = self.accuracy(logits, y)

        self.log("ptl/train_loss", loss)
        self.log("ptl/train_accuracy", accuracy)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        accuracy = self.accuracy(logits, y)
        return {"val_loss": loss, "val_accuracy": accuracy}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_acc = torch.stack([x["val_accuracy"] for x in outputs]).mean()
        self.log("ptl/val_loss", avg_loss)
        self.log("ptl/val_accuracy", avg_acc)

    @staticmethod
    def download_data(data_dir):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307, ), (0.3081, ))
        ])
        with FileLock(os.path.expanduser("~/.data.lock")):
            return MNIST(data_dir, train=True, download=True, transform=transform)

    def prepare_data(self):
        mnist_train = self.download_data(self.data_dir)

        self.mnist_train, self.mnist_val = random_split(
            mnist_train, [55000, 5000])

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=int(self.batch_size))

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=int(self.batch_size))

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer


def train_mnist(config):
    model = LightningMNISTClassifier(config)
    trainer = pl.Trainer(max_epochs=10, enable_progress_bar=True)


    trainer.fit(model)

In [8]:
def train_mnist_no_tune():
    config = {
        "layer_1_size": 128,
        "layer_2_size": 256,
        "lr": 1e-3,
        "batch_size": 64
    }
    train_mnist(config)

In [9]:
train_mnist_no_tune()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33.0 K
2 | layer_3 | Linear | 2.6 K 
-----------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [10]:
from pytorch_lightning.loggers import TensorBoardLogger
from ray import air, tune
from ray.air import session
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.tune.integration.pytorch_lightning import TuneReportCallback, \
    TuneReportCheckpointCallback

In [ ]:
TuneReportCallback(
    {
        "loss": "ptl/val_loss",
        "mean_accuracy": "ptl/val_accuracy"
    },
    on="validation_end")

In [11]:
def train_mnist_tune(config, num_epochs=10, num_gpus=0, data_dir="~/data"):
    data_dir = os.path.expanduser(data_dir)
    model = LightningMNISTClassifier(config, data_dir)
    trainer = pl.Trainer(
        max_epochs=num_epochs,
        # If fractional GPUs passed in, convert to int.
        gpus=math.ceil(num_gpus),
        logger=TensorBoardLogger(
            save_dir=os.getcwd(), name="", version="."),
        enable_progress_bar=False,
        callbacks=[
            TuneReportCallback(
                {
                    "loss": "ptl/val_loss",
                    "mean_accuracy": "ptl/val_accuracy"
                },
                on="validation_end")
        ])
    trainer.fit(model)

In [12]:
config = {
    "layer_1_size": tune.choice([32, 64, 128]),
    "layer_2_size": tune.choice([64, 128, 256]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([32, 64, 128]),
}

In [13]:
num_epochs = 10

scheduler = ASHAScheduler(
    max_t=num_epochs,
    grace_period=1,
    reduction_factor=2)

In [14]:
reporter = CLIReporter(
    parameter_columns=["layer_1_size", "layer_2_size", "lr", "batch_size"],
    metric_columns=["loss", "mean_accuracy", "training_iteration"])

In [15]:
gpus_per_trial = 0
data_dir = "~/data"

train_fn_with_parameters = tune.with_parameters(train_mnist_tune,
                                                num_epochs=num_epochs,
                                                num_gpus=gpus_per_trial,
                                                data_dir=data_dir)

In [16]:
resources_per_trial = {"cpu": 1, "gpu": gpus_per_trial}

In [17]:
def tune_mnist_asha(num_samples=10, num_epochs=10, gpus_per_trial=0, data_dir="~/data"):
    config = {
        "layer_1_size": tune.choice([32, 64, 128]),
        "layer_2_size": tune.choice([64, 128, 256]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([32, 64, 128]),
    }

    scheduler = ASHAScheduler(
        max_t=num_epochs,
        grace_period=1,
        reduction_factor=2)

    reporter = CLIReporter(
        parameter_columns=["layer_1_size", "layer_2_size", "lr", "batch_size"],
        metric_columns=["loss", "mean_accuracy", "training_iteration"])

    train_fn_with_parameters = tune.with_parameters(train_mnist_tune,
                                                    num_epochs=num_epochs,
                                                    num_gpus=gpus_per_trial,
                                                    data_dir=data_dir)
    resources_per_trial = {"cpu": 1, "gpu": gpus_per_trial}
    
    tuner = tune.Tuner(
        tune.with_resources(
            train_fn_with_parameters,
            resources=resources_per_trial
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        run_config=air.RunConfig(
            name="tune_mnist_asha",
            progress_reporter=reporter,
        ),
        param_space=config,
    )
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)

In [19]:
tune_mnist_asha()

2023-03-07 18:04:37,419	INFO worker.py:1553 -- Started a local Ray instance.


== Status ==
Current time: 2023-03-07 18:04:39 (running for 00:00:00.21)
Memory usage on this node: 1.9/12.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/2 CPUs, 0/0 GPUs, 0.0/7.37 GiB heap, 0.0/3.68 GiB objects
Result logdir: /root/ray_results/tune_mnist_asha
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------------+----------+------------------+----------------+----------------+-------------+--------------+
| Trial name                   | status   | loc              |   layer_1_size |   layer_2_size |          lr |   batch_size |
|------------------------------+----------+------------------+----------------+----------------+-------------+--------------|
| train_mnist_tune_86f06_00000 | RUNNING  | 172.28.0.12:6139 |             32 |             64 | 0.000141977 |          128 |
| train_mnist_tune_86f06_00001 | PENDING  |                  |             64 |     

(train_mnist_tune pid=6139) /usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
(train_mnist_tune pid=6139)   rank_zero_deprecation(
(train_mnist_tune pid=6139) GPU available: False, used: False
(train_mnist_tune pid=6139) TPU available: False, using: 0 TPU cores
(train_mnist_tune pid=6139) IPU available: False, using: 0 IPUs
(train_mnist_tune pid=6139) HPU available: False, using: 0 HPUs


(train_mnist_tune pid=6139) Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
(train_mnist_tune pid=6139) Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to /root/data/MNIST/raw/train-images-idx3-ubyte.gz


100%|██████████| 9912422/9912422 [00:00<00:00, 46072783.50it/s]


...

== Status ==
Current time: 2023-03-07 18:11:53 (running for 00:07:14.22)
Memory usage on this node: 2.9/12.7 GiB 
Using AsyncHyperBand: num_stopped=7
Bracket: Iter 8.000: -0.1616043671965599 | Iter 4.000: -0.23117558658123016 | Iter 2.000: -0.2633167505264282 | Iter 1.000: -0.31370092928409576
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.37 GiB heap, 0.0/3.68 GiB objects
Current best trial: 86f06_00002 with loss=0.14058569073677063 and parameters={'layer_1_size': 32, 'layer_2_size': 64, 'lr': 0.0009660718077979762, 'batch_size': 128}
Result logdir: /root/ray_results/tune_mnist_asha
Number of trials: 10/10 (1 PENDING, 2 RUNNING, 7 TERMINATED)
+------------------------------+------------+------------------+----------------+----------------+-------------+--------------+----------+-----------------+----------------------+
| Trial name                   | status     | loc              |   layer_1_size |   layer_2_size |          lr |   batch_size |     loss |   mean_accuracy |   trainin

(train_mnist_tune pid=6139) GPU available: False, used: False
(train_mnist_tune pid=6139) TPU available: False, using: 0 TPU cores
(train_mnist_tune pid=6139) IPU available: False, using: 0 IPUs
(train_mnist_tune pid=6139) HPU available: False, using: 0 HPUs
(train_mnist_tune pid=6139) 
(train_mnist_tune pid=6139)   | Name    | Type   | Params
(train_mnist_tune pid=6139) -----------------------------------
(train_mnist_tune pid=6139) 0 | layer_1 | Linear | 25.1 K
(train_mnist_tune pid=6139) 1 | layer_2 | Linear | 8.4 K 
(train_mnist_tune pid=6139) 2 | layer_3 | Linear | 2.6 K 
(train_mnist_tune pid=6139) -----------------------------------
(train_mnist_tune pid=6139) 36.1 K    Trainable params
(train_mnist_tune pid=6139) 0         Non-trainable params
(train_mnist_tune pid=6139) 36.1 K    Total params
(train_mnist_tune pid=6139) 0.145     Total estimated model params size (MB)


== Status ==
Current time: 2023-03-07 18:15:21 (running for 00:10:42.70)
Memory usage on this node: 2.7/12.7 GiB 
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 8.000: -0.15791214257478714 | Iter 4.000: -0.17471830546855927 | Iter 2.000: -0.2366371899843216 | Iter 1.000: -0.29755789041519165
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.37 GiB heap, 0.0/3.68 GiB objects
Current best trial: 86f06_00007 with loss=0.13429000973701477 and parameters={'layer_1_size': 32, 'layer_2_size': 128, 'lr': 0.0006869937718490195, 'batch_size': 64}
Result logdir: /root/ray_results/tune_mnist_asha
Number of trials: 10/10 (2 RUNNING, 8 TERMINATED)
+------------------------------+------------+------------------+----------------+----------------+-------------+--------------+----------+-----------------+----------------------+
| Trial name                   | status     | loc              |   layer_1_size |   layer_2_size |          lr |   batch_size |     loss |   mean_accuracy |   training_iteratio

2023-03-07 18:15:56,276	INFO tune.py:798 -- Total run time: 677.37 seconds (677.19 seconds for the tuning loop).


== Status ==
Current time: 2023-03-07 18:15:56 (running for 00:11:17.21)
Memory usage on this node: 2.0/12.7 GiB 
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 8.000: -0.15791214257478714 | Iter 4.000: -0.17471830546855927 | Iter 2.000: -0.2366371899843216 | Iter 1.000: -0.31370092928409576
Resources requested: 0/2 CPUs, 0/0 GPUs, 0.0/7.37 GiB heap, 0.0/3.68 GiB objects
Current best trial: 86f06_00007 with loss=0.13432417809963226 and parameters={'layer_1_size': 32, 'layer_2_size': 128, 'lr': 0.0006869937718490195, 'batch_size': 64}
Result logdir: /root/ray_results/tune_mnist_asha
Number of trials: 10/10 (10 TERMINATED)
+------------------------------+------------+------------------+----------------+----------------+-------------+--------------+----------+-----------------+----------------------+
| Trial name                   | status     | loc              |   layer_1_size |   layer_2_size |          lr |   batch_size |     loss |   mean_accuracy |   training_iteration |
|------